In [ ]:
import json
import pandas as pd

In [ ]:
path = "/data/codes/prep_ps_pykaldi/prep_data/jsonl_v1/info_in_domain_long_sentence_testset_old.jsonl"
with open(path, "r", encoding="utf-8") as f:
    lines = [json.loads(line.strip()) for line in f.readlines()]

df = pd.DataFrame(lines)
df.head()

In [ ]:
df[["id", "text"]].to_csv("/data/codes/prep_ps_pykaldi/prep_data/info_in_domain_long_sentence_testset_old.csv", sep="|", index=None, header=None)

#### audio processing

In [ ]:
import pandas as pd
import torchaudio
import librosa
import shutil
import json
import os

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=True)

In [ ]:
audio_dir = "/data/codes/prep_ps_pykaldi/prep_data/wav"
audio_files = os.listdir(audio_dir)
audio_files = [audio_file.split(".")[0] for audio_file in audio_files]
audio_files = set(audio_files)

In [ ]:
path = "/data/codes/prep_ps_pykaldi/prep_data/info_in_domain_long_sentence_testset.csv"
df = pd.read_csv(path, names=["id", "text"], sep="|")
df.head()

In [ ]:
df.id.parallel_apply(lambda x: str(x) in audio_files).sum()

In [ ]:
in_dir = "/data/audio_data/prep_submission_audio/10"
out_dir = audio_dir

def copy_audio(id):
    path = f'{in_dir}/{id}.wav'
    wav, sr = torchaudio.load(path)
    if sr == 8000:
        return False
    return True
    

is_success = df.id.parallel_apply(copy_audio)

In [ ]:
df[~is_success]